In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import dask
dask.__version__

'2022.03.0'

In [3]:
xgb.__version__

'1.6.1'

In [4]:
!nvidia-smi

Mon May 16 14:04:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   31C    P8    28W / 280W |      5MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0 Off |                  N/A |
| 41%   

In [5]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [6]:
train_x = dd.read_csv('../input/xgtrain.csv')
test_x = dd.read_csv('../input/xgval.csv')

In [7]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [8]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=["GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f",
                                                 "GPU-50aeb092-88f5-4e0b-7f73-32741666f319"])
client = Client(cluster)


2022-05-16 14:04:48,388 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-16 14:04:48,394 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [9]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [10]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)

In [11]:
num_round = 1000

In [21]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softmax']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3, 10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 2, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'eval_metric': trial.suggest_categorical('eval_metric',['merror']),
        'num_class': 10,

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    acc = accuracy_score(test_y, predictions)
    
    return acc

In [22]:
study = optuna.create_study(direction='maximize')

[I 2022-05-16 14:13:59,462] A new study created in memory with name: no-name-50e80f5f-2857-4db8-a053-2b95f3c92798


In [23]:
%%time
study.optimize(objective, n_trials=2)

[14:14:00] task [xgboost.dask]:tcp://127.0.0.1:41775 got new rank 0
[14:14:00] task [xgboost.dask]:tcp://127.0.0.1:45717 got new rank 1
[I 2022-05-16 14:14:20,464] Trial 0 finished with value: 0.8007142857142857 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 0.1923775268189964, 'alpha': 0.33594703703369716, 'colsample_bytree': 0.46700490176040155, 'subsample': 0.9411945915990364, 'learning_rate': 0.0016194177849045126, 'max_depth': 2, 'min_child_weight': 238, 'eval_metric': 'merror'}. Best is trial 0 with value: 0.8007142857142857.
[14:14:20] task [xgboost.dask]:tcp://127.0.0.1:41775 got new rank 0
[14:14:20] task [xgboost.dask]:tcp://127.0.0.1:45717 got new rank 1
[I 2022-05-16 14:24:53,178] Trial 1 finished with value: 0.8563095238095239 and parameters: {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'lambda': 4.716807716607245, 'alpha': 8.845853499502253, 'colsample_bytree': 0.9639282539347016, 'subsample': 0.5668230954447968, 'learning

CPU times: user 16 s, sys: 8.39 s, total: 24.4 s
Wall time: 10min 52s
